In [7]:
import tkinter as tk
from tkinter import messagebox
from sqlalchemy import create_engine, text

# Configuração de conexão com o banco de dados
DB_USER = 'postgres'
DB_PASSWORD = '1010'
DB_HOST = 'localhost'
DB_PORT = '5432'
DB_NAME = 'DB_Final'

def connect_db():
    try:
        conexao_bd = f'postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}'
        engine = create_engine(
            conexao_bd,
            connect_args={'options': '-csearch_path=sistema,public'}
        )
        conn = engine.connect()
        return conn
    except Exception as e:
        print(f"Erro ao conectar ao banco de dados: {e}")
        return None
    

def buscar_por_cpf_exclusao(cpf):
    conn = connect_db()
    if conn:
        try:
            # Buscar informações da pessoa pelo CPF
            pessoa_query = text("""
                SELECT p.ID_Pessoa, p.Nome, p.CPF 
                FROM Pessoa p 
                WHERE p.CPF = :cpf
            """)
            pessoa_result = conn.execute(pessoa_query, {"cpf": cpf}).fetchone()

            if pessoa_result:
                id_pessoa, nome, cpf = pessoa_result

                # Verificar se a pessoa é Médico, Secretário ou Cliente
                medico_query = text("SELECT CRM, Especializacao FROM Medico WHERE ID_Pessoa = :id_pessoa")
                medico_result = conn.execute(medico_query, {"id_pessoa": id_pessoa}).fetchone()

                cliente_query = text("SELECT Convenio FROM Cliente WHERE ID_Pessoa = :id_pessoa")
                cliente_result = conn.execute(cliente_query, {"id_pessoa": id_pessoa}).fetchone()

                secretario_query = text("SELECT ID_Secretario FROM Secretario WHERE ID_Pessoa = :id_pessoa")
                secretario_result = conn.execute(secretario_query, {"id_pessoa": id_pessoa}).fetchone()

                # Exibe as informações da pessoa
                resultado = f"Nome: {nome}\nCPF: {cpf}\n"
                if medico_result:
                    crm, especializacao = medico_result
                    resultado += f"Médico\nCRM: {crm}\nEspecialização: {especializacao}\n"
                    tipo = "Médico"
                elif cliente_result:
                    convenio = cliente_result[0]
                    resultado += f"Cliente\nConvênio: {convenio}\n"
                    tipo = "Cliente"
                elif secretario_result:
                    resultado += "Secretário\n"
                    tipo = "Secretário"
                else:
                    messagebox.showerror("Erro", "Essa pessoa não é médico, cliente ou secretário.")
                    return

                # Atualiza o campo de resultado
                text_resultado.config(state='normal')
                text_resultado.delete(1.0, tk.END)
                text_resultado.insert(tk.END, resultado)
                text_resultado.config(state='disabled')

                # Exibe os botões de exclusão e cancelamento
                btn_excluir.config(state='normal', command=lambda: excluir(id_pessoa, tipo))
                btn_cancelar.config(state='normal')
            else:
                messagebox.showerror("Erro", "CPF não encontrado.")
        except Exception as e:
            messagebox.showerror("Erro", f"Ocorreu um erro ao buscar o CPF: {e}")
            print(f"Erro ao buscar por CPF: {str(e)}")
        finally:
            conn.close()

def excluir(id_pessoa, tipo):
    conn = connect_db()
    if conn:
        try:
            # Definir a tabela de exclusão com base no tipo
            if tipo == "Médico":
                exclusao_query = text("DELETE FROM Medico WHERE ID_Pessoa = :id_pessoa")
                exclusao_tabela = "Médico"
            elif tipo == "Secretário":
                exclusao_query = text("DELETE FROM Secretario WHERE ID_Pessoa = :id_pessoa")
                exclusao_tabela = "Secretário"
            elif tipo == "Cliente":
                exclusao_query = text("DELETE FROM Cliente WHERE ID_Pessoa = :id_pessoa")
                exclusao_tabela = "Cliente"
            else:
                messagebox.showerror("Erro", "Tipo de exclusão inválido.")
                return

            # Realiza a exclusão na tabela específica
            conn.execute(exclusao_query, {"id_pessoa": id_pessoa})

            # Após excluir da tabela específica, excluir da tabela Pessoa
            pessoa_exclusao_query = text("DELETE FROM Pessoa WHERE ID_Pessoa = :id_pessoa")
            conn.execute(pessoa_exclusao_query, {"id_pessoa": id_pessoa})

            conn.commit()
            messagebox.showinfo("Sucesso", f"{exclusao_tabela} e os dados da Pessoa excluídos com sucesso!")
            root.destroy()  # Fecha a janela após exclusão
        except Exception as e:
            conn.rollback()
            messagebox.showerror("Erro", f"Erro ao excluir: {e}")
        finally:
            conn.close()

# Função para capturar o CPF e realizar a busca
def buscar():
    cpf = entry_cpf.get()
    if cpf:
        buscar_por_cpf_exclusao(cpf)
    else:
        messagebox.showerror("Erro", "Por favor, insira um CPF válido.")

# Interface gráfica usando Tkinter
root = tk.Tk()
root.title("Exclusão de Médico, Secretário ou Cliente")

# Campo de texto para o CPF
tk.Label(root, text="CPF").grid(row=0, column=0)
entry_cpf = tk.Entry(root)
entry_cpf.grid(row=0, column=1)

# Botão para buscar o CPF
btn_buscar = tk.Button(root, text="Buscar", command=buscar)
btn_buscar.grid(row=0, column=2)

# Campo de texto para exibir as informações
text_resultado = tk.Text(root, height=10, width=60, state='disabled')
text_resultado.grid(row=1, column=0, columnspan=3)

# Botão de excluir (inicialmente desativado)
btn_excluir = tk.Button(root, text="Excluir", state='disabled')
btn_excluir.grid(row=2, column=1)

# Botão de cancelar (inicialmente desativado)
btn_cancelar = tk.Button(root, text="Cancelar", state='disabled', command=root.destroy)
btn_cancelar.grid(row=2, column=2)

# Loop da interface gráfica
root.mainloop()
